In [ ]:
from torch import Tensor
from torch import flatten
from torch.nn import Module
from torch.nn import Linear, Dropout, ReLU

def select_activation(name: str):
    if name == 'relu':
        return ReLU()

class MLP(Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, p: float, activation: str = 'relu'):
        super().__init__()
        self.input_layer = Linear(input_size, hidden_size)
        self.activation = select_activation(activation)
        self.dropout = Dropout(p=p)
        self.output_layer =Linear(hidden_size, output_size)

    def forward(self, sequence: Tensor) -> Tensor:
        sequence = self.input_layer(flatten(sequence, 1))
        sequence = self.activation(sequence)
        sequence = self.dropout(sequence)
        return self.output_layer(sequence)

In [ ]:
from torch.utils.data import Dataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

class Digits(Dataset):
    def __init__(self, train: bool):
        self.data = MNIST(root='data/datasets', download=True, train=train, transform=ToTensor())

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
from mlbus.torch import Aggregate
from mlbus.torch import Loader, Criterion, Optimizer
from typing import Callable
from torch.optim import Optimizer
from torch import inference_mode

class Classifier(Aggregate):
    def __init__(self, hash: str, model: Module, criterion: Criterion):
        super().__init__(hash)
        self.model = model
        self.criterion = criterion

    def forward(self, input: Tensor) -> Tensor:
        return self.model(input)

    def fit(self, loader: Loader, callback: Callable, optimizer: Optimizer):
        for batch, (input, target) in enumerate(loader, start=1):
            optimizer.zero_grad()
            output = self.model(input)
            loss = self.criterion(output, target)
            loss.backward()
            optimizer.step()
            callback(batch, loss.item(), output, target)

    @inference_mode
    def evaluate(self, loader: Loader, callback: Callable):
        for batch, (input, target) in enumerate(loader, start=1):
            output = self.model(input)
            loss = self.criterion(output, target)
            callback(batch, loss.item(), output, target)

In [ ]:
from mlbus.torch import Repository
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

Repository.models.register(MLP)
Repository.criterions.register(CrossEntropyLoss)
Repository.optimizers.register(Adam)
Repository.datasets.register(Digits)

In [ ]:
from mlbus import Dispatcher
from mlbus.torch import Loaders
from mlbus.torch.callbacks import Loss, Accuracy

model = MLP(784, 256, 10, p=0.5, activation='relu')
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
loaders = Loaders()
loaders.add('train', Digits(train=True), batch_size=256, shuffle=True)
loaders.add('evaluation', Digits(train=False), batch_size=256, shuffle=True)
repository = Repository()
classifier = Classifier(model, criterion, optimizer)
callback = Dispatcher(Loss(), Accuracy())

NameError: name 'MLP' is not defined